In [28]:
import pandas as pd
import numpy as np
import os.path as op
import os
import seaborn as sns
import glob
from PIL import Image
# import imagehash

In [29]:
conditions_file_path = 'conditions_file.csv'


In [30]:
# df_set_cols = df.columns

In [31]:
# fixed_columns = df[['trial', 'attend', 'reliability', 'validity', 'test_item', 'stim_side']]

# other_columns = df.drop(columns=['trial', 'attend', 'reliability', 'validity', 'test_item', 'stim_side'])

# shuffled_other_columns = other_columns.sample(frac=1).reset_index(drop=True)

# result_df = pd.concat([fixed_columns, shuffled_other_columns], axis=1)

# df = result_df

In [32]:
# df = df.drop(columns = ['IT_diff', 'V2_diff', 'IT_bin', 'V2_bin', 'nonabs_IT_diff', 'nonabs_V2_diff',	'Image1', 'Image2',	'IT_root_im1',	'IT_root_im2',	'IT_im1_im2',	'V2_root_im1',	'V2_root_im2',	'V2_im1_im2',	'cat_root',	'cat_im1',	'cat_im2',	'V2_im2_minus_im1',	'IT_im2_minus_im1',	'IT_most_similar',	'V2_most_similar'])
#'concept_root',	'concept_im1',	'concept_im2', don't exist anymore

In [33]:
### check assignment of probe_direction, cue_img

In [34]:
def conds_shuffling(conditions_file_path, b):
    # Read the conditions file into a DataFrame
    df = pd.read_csv(conditions_file_path)

    df_set_cols = df.columns
    fixed_columns = df[['trial', 'attend', 'reliability', 'validity', 'test_item', 'stim_side']]

    other_columns = df.drop(columns=['trial', 'attend', 'reliability', 'validity', 'test_item', 'stim_side'])

    shuffled_other_columns = other_columns.sample(frac=1).reset_index(drop=True)

    result_df = pd.concat([fixed_columns, shuffled_other_columns], axis=1)

    df = result_df
#     IT_bin, V2_bin , 'nonabs_IT_diff', 'nonabs_V2_diff',
    df = df.drop(columns = ['IT_diff', 'V2_diff', 	'Image1', 'Image2',	'IT_root_im1',	'IT_root_im2',	'IT_im1_im2',	'V2_root_im1',	'V2_root_im2',	'V2_im1_im2',	'cat_root',	'cat_im1',	'cat_im2',	'V2_im2_minus_im1',	'IT_im2_minus_im1',	'IT_most_similar',	'V2_most_similar'])
    n_stims = 6 #number of choice stims
    choice_radius = .35 #radius of circle to array the stims around
    ntrials = 300

    ### Updating right_cue_length, left_cue_length, right_cue_offset, left_cue_offset, attend_direction, rec_length - do this in the end
    # side = [1]*int(ntrials/2) + [-1]*int(ntrials/2)

    # np.random.shuffle(side)

    # df['stim_side'] = side

    rec_length = .33  #reliability cue rectangle length

    reliability_to_rec_length = {round(5/6,2):11/12,
                                round(1/6,2):1/12,
                                round(2/3,2):7/12,
                                round(1/3,2):5/12}

    attend_direction = []
    right_cue_opacity = []
    left_cue_opacity = []
    left_img = []
    right_img = []


    for i in range(len(df)):
        side = df.loc[i, 'stim_side']
        img1 = df.loc[i, 'img1']
        img2 = df.loc[i, 'img2']

        if side == 1:
            left_img.append(img1)
            right_img.append(img2)
        else:
            left_img.append(img2)
            right_img.append(img1)

    df['left_img'] = left_img
    df['right_img'] = right_img

    attend_direction = []
    right_cue_opacity = []
    left_cue_opacity = []

    for i in range(len(df)):
        attend = df.loc[i, 'attend']
        side = df.loc[i, 'stim_side']
        reliability = df.loc[i, 'reliability']

        if attend == 'img2':
            if side == 1:
                attend_direction.append('right')
                right_cue_opacity.append(reliability_to_rec_length[round(reliability, 2)])
                left_cue_opacity.append(reliability_to_rec_length[round(1 - reliability, 2)])
            else:
                attend_direction.append('left')
                right_cue_opacity.append(reliability_to_rec_length[round(1 - reliability, 2)])
                left_cue_opacity.append(reliability_to_rec_length[round(reliability, 2)])
        elif attend == 'img1':
            if side == 1:
                attend_direction.append('left')
                right_cue_opacity.append(reliability_to_rec_length[round(1 - reliability, 2)])
                left_cue_opacity.append(reliability_to_rec_length[round(reliability, 2)])
            else:
                attend_direction.append('right')
                right_cue_opacity.append(reliability_to_rec_length[round(reliability, 2)])
                left_cue_opacity.append(reliability_to_rec_length[round(1 - reliability, 2)])



    df['right_cue_length'] = np.array(right_cue_opacity) * rec_length/2.0
    df['left_cue_length'] = np.array(left_cue_opacity) * rec_length/2.0
    df['right_cue_offset'] = [x/2 for x in df['right_cue_length']]
    df['left_cue_offset'] = [-1*x/2 for x in df['left_cue_length']]
    df['attend_direction'] = attend_direction 
    df['rec_length'] = rec_length





    #probe stim locations

    position = []
    for i in range(n_stims + 2):

        theta = i*(2*np.pi/(n_stims + 2)) #angle
        position.append((np.round(choice_radius *np.cos(theta)*1.5,2),
                   np.round(choice_radius *np.sin(theta),2)))

    position = [x for x in position if x[1] != 0] #drop left and right stimuli
    for i in range(ntrials):
        np.random.shuffle(position)

        df.loc[i,'mem_x'] = position[0][0]
        df.loc[i,'mem_y'] = position[0][1]

        df.loc[i,'dismem_x'] = position[1][0]
        df.loc[i,'dismem_y'] = position[1][1]

        #now do distractors
        for j in range(n_stims - 2):
            df.loc[i,'dis' + str(j) + '_x'] = position[j+2][0]
            df.loc[i,'dis' + str(j) + '_y'] = position[j+2][1]


    #set memory stimulus
    mem_stim = []
    distractor_stim = []
    for i in range(len(df)):
        im1 = df.loc[i, 'img1']
        im2 = df.loc[i, 'img2']
        test_item = df.loc[i, 'test_item']

        if test_item == 'img1':
            mem_stim.append(im1)
            distractor_stim.append(im2)
        elif test_item == 'img2':
            mem_stim.append(im2)
            distractor_stim.append(im1)

    df['mem_stim'] = mem_stim
    df['distractor_stim'] = distractor_stim


    #get test item direction
    probe_direction = []
    cue_img = []
    for i in range(len(df)):
        probe = df.loc[i, 'test_item']
        side = df.loc[i, 'stim_side']

        if probe == 'img1':
            if side == 1:
                probe_direction.append('left')
                cue_img.append('./stimuli/cue_left.png')
            else:
                probe_direction.append('right')
                cue_img.append('./stimuli/cue_right.png')
        elif probe == 'img2':
            if side == 1:
                probe_direction.append('right')
                cue_img.append('./stimuli/cue_right.png')
            else:
                probe_direction.append('left')
                cue_img.append('./stimuli/cue_left.png')


    df['probe_direction'] = probe_direction    
    df['cue_img'] = cue_img    
    #all these come from triplets IT_diff	V2_diff	root	Image1	Image2	IT_root_im1	IT_root_im2	IT_im1_im2	V2_root_im1	V2_root_im2	V2_im1_im2	cat_root	cat_im1	cat_im2	V2_im2_minus_im1	IT_im2_minus_im1	IT_most_similar	V2_most_similar
    triplets = pd.read_csv('triplets.csv')
    df['root'] = [x.split('/')[-1] for x in df['ping_img']]
    df = pd.merge(df, 
            triplets,
            on = ['root'])
    condition_string = 'conditions_file_'+str(b)
    df['cond_file'] = condition_string
    df.to_csv('./resources/' + condition_string + '.csv',
                       index = False)

In [35]:
i = 0
while i < 11:
#     some_string = "conditions_" + str(i)
#     print(some_string)
    conds_shuffling(conditions_file_path, str(i))
    i += 1

In [36]:
# df = df.sample(frac=1) #randomly shuffle



In [ ]:
#### one way to do this is track down the dates of the gitlab 
###conditions file updates and manually batch these

In [ ]:
### another way is to check the conditions file used per participant 
###and if the data there is different from the previous one update the batch
###the problem is there might not be enough conditions files in the batch used
### and we wont be able to absolutely accurately separate batches